In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import urllib.request, json
import requests
from requests import Request, Session
from requests.auth import HTTPBasicAuth
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from datetime import datetime
import datetime as dt
import pytz
from pandas import * 
from dateutil import tz
import warnings
warnings.filterwarnings("ignore")
import datetime as dt
import pandas as pd

In [ ]:
fmpapi='*' #input api key
def fmp_data(ticker): #Daily data
  with urllib.request.urlopen('https://financialmodelingprep.com/api/v3/historical-price-full/'+ticker+'?from='+start+'&to='+stop+'&apikey='+fmpapi) as url:
    data = json.loads(url.read().decode())
    df = pd.DataFrame(json_normalize(data['historical']))
    df['symbol'] = ticker
    return df
def get_live_price(ticker): #Daily data
  with urllib.request.urlopen('https://financialmodelingprep.com/api/v3/quote-short/'+ticker+'?apikey='+fmpapi) as url:
    data = json.loads(url.read().decode())
    df = pd.DataFrame(json_normalize(data))
    return df['price']

In [ ]:
def get_price(ticker):
    try:
        return np.array(get_live_price(ticker))
    except:
        print(f"{ticker.upper()} is not a valid ticker")
        
def get_data(ticker, start, end):
    try:
        with urllib.request.urlopen('https://financialmodelingprep.com/api/v3/historical-price-full/'+ticker+'?from='+start+'&to='+end+'&apikey='+fmpapi) as url:
            data = json.loads(url.read().decode())
            df = pd.DataFrame(json_normalize(data['historical']))
            df['symbol'] = ticker
            return df["adjClose"]
    except:
        print(f'No data found for {ticker}.')

In [53]:
class Ledger:
    def __init__(self):
        self.transactions = {}
        self.holdings = {"USD": 0}
        
    def get_holdings(self):
        return self.holdings
    
    def get_transactions(self):
        return self.transactions

    def register_transaction(self, transaction, symbol, shares, date):
        cost = np.array(
            get_data(
                symbol, 
                date,
                dt.datetime.now().date().strftime("%Y-%m-%d")).values[-1])
        total = cost*shares
        transaction_id = len(ledger.get_transactions())
        self.transactions.update({
              transaction_id: {
                    "type" : transaction,
                    "date": date,
                    "symbol": symbol.upper(),
                    "shares": shares,
                    "cost": cost,
                    "total" : total,
                } 
            })
        holdings = self.get_holdings()
        if transaction == "buy":
            if symbol in holdings.keys():
                holding = holdings[symbol]
                holdings["USD"] -= total if holdings["USD"] > total else holdings["USD"]
                holding["cost"] = (
                    (holding["cost"]*holding["shares"] + total) / 
                    (holding["shares"] + shares))
                holding["shares"] += shares
            else:
                holdings.update({
                    symbol: {
                        "shares": shares,
                        "cost": cost
                    }
                })
        if transaction == "sell":
            holding = holdings[symbol]
            holdings["USD"] += total
            holding["shares"] -= shares
            if holding["shares"] == 0:
                del holdings[symbol]

In [54]:
def get_ts(self):
    def get_portfolio(portfolio, start, end):
        series = pd.Series(dtype="float64")
        for symbol in portfolio.keys():
            s = get_data(symbol, start, end) * portfolio[symbol]
            if symbol == "USD":
                continue
            series = series.add(s, fill_value = 0)
        return series + portfolio["USD"]
    transactions = self.get_transactions()
    dates = [transactions[t]["date"] for t in transactions] + [dt.datetime.now().date().strftime("%Y-%m-%d")]
    portfolio = {"USD": 0}
    series = pd.Series(dtype="float64")
    for t in transactions:
        tr = transactions[t]
        adj = 0
        if tr["type"] == "buy":
            adj = -1*(tr["total"] if portfolio["USD"] > tr["total"]
            else portfolio["USD"])
            if tr["symbol"] in portfolio:
                portfolio[tr["symbol"]] += tr["shares"] 
            else:
                portfolio.update({tr["symbol"] : tr["shares"]})
        if tr["type"] == "sell":
            portfolio["USD"] += tr["total"]
            portfolio[tr["symbol"]] -= tr["shares"] 
        add = (tr["total"] - portfolio["USD"]) if tr["total"] > portfolio["USD"] else 0
        portfolio["USD"] += adj
        s = get_portfolio(portfolio, dates[t], dates[t+1])
        series = (series[:s.index[0]] + add).append(s) 
    return pd.Series([series.loc[x] 
         if isinstance(series.loc[x], np.float64) 
         else series.loc[x].values[0] for x in series.index.unique()],
                     index = series.index.unique())

In [55]:
ledger = Ledger()
ledger.register_transaction("buy", "TSLA", 100, "2022-01-01")
ledger.register_transaction("buy", "AAPL", 100, "2022-02-01")
ledger.register_transaction("sell", "TSLA", 50, "2022-02-01")
ledger.register_transaction("sell", "AAPL", 50, "2022-03-01")
holdings = ledger.get_holdings()
df = pd.DataFrame(holdings).transpose()
df["cost"].loc["USD"] = 1
df["current_price"] = np.array([df.cost.loc["USD"]] + [get_price(x) for x in df.index[1:]])
df["total market value"] = np.array(df.current_price * df.shares)
df["return rate"] = np.array((df.current_price - df.cost)/ df.cost)
df["P&L"] = np.array(df.current_price - df.cost)

In [56]:
df.head()

,shares,cost,current_price,total market value,return rate,P&L
USD,54722.49985,1,1,54722.49985,0.0,0
TSLA,50,1199.780029,[766.37],[38318.5],[-0.36124124299788624],[-433.410029]
AAPL,50,174.387817,[150.62],[7531.0],[-0.1362928753216746],[-23.767817000000008]


In [ ]:
transactions = ledger.get_transactions()
df = pd.DataFrame(transactions).transpose()
df.loc[df.type == "sell", "total"] = df.loc[df.type == "sell", "total"] * -1

In [ ]:
df.head()

,type,date,symbol,shares,cost,total
0,buy,2022-01-01,TSLA,100,795.35,79535.0
1,buy,2022-02-01,AAPL,100,154.73,15473.0
2,sell,2022-03-01,AAPL,50,154.73,-7736.5
3,sell,2022-03-01,TSLA,50,795.35,-39767.5
